<a href="https://colab.research.google.com/github/Navdeeposm19/Deepfake-voice-catcher-/blob/main/project_star.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("awsaf49/asvpoof-2019-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'asvpoof-2019-dataset' dataset.
Path to dataset files: /kaggle/input/asvpoof-2019-dataset


In [ ]:
import os
from pathlib import Path

ROOT = Path("/kaggle/input/asvpoof-2019-dataset/LA/LA")

print("LA root exists:", ROOT.exists())
print("Contents of LA root:")
for x in ROOT.iterdir():
    print("  ", x.name)


LA root exists: True
Contents of LA root:
   ASVspoof2019_LA_dev
   ASVspoof2019_LA_train
   ASVspoof2019_LA_cm_protocols
   ASVspoof2019_LA_eval
   ASVspoof2019_LA_asv_protocols
   README.LA.txt
   ASVspoof2019_LA_asv_scores


In [ ]:
from pathlib import Path

ROOT = Path("/kaggle/input/asvpoof-2019-dataset/LA/LA")

TRAIN_AUDIO = ROOT / "ASVspoof2019_LA_train" / "flac"
DEV_AUDIO   = ROOT / "ASVspoof2019_LA_dev" / "flac"
EVAL_AUDIO  = ROOT / "ASVspoof2019_LA_eval" / "flac"

print("Train audio exists:", TRAIN_AUDIO.exists())
print("Dev audio exists:", DEV_AUDIO.exists())
print("Eval audio exists:", EVAL_AUDIO.exists())


Train audio exists: True
Dev audio exists: True
Eval audio exists: True


In [ ]:
import librosa
import numpy as np

TARGET_SR = 16000
FIXED_SEC = 4
FIXED_LEN = TARGET_SR * FIXED_SEC

def preprocess_audio(path):
    y, sr = librosa.load(path, sr=TARGET_SR, mono=True)

    y, _ = librosa.effects.trim(y, top_db=25)

    if np.max(np.abs(y)) > 0:
        y = y / np.max(np.abs(y))

    if len(y) < FIXED_LEN:
        y = np.pad(y, (0, FIXED_LEN - len(y)))
    else:
        y = y[:FIXED_LEN]

    return y


In [ ]:
import glob

sample_file = glob.glob(str(TRAIN_AUDIO / "*.flac"))[0]
print(sample_file)

y = preprocess_audio(sample_file)
print("Length:", len(y))
print("Min/Max:", y.min(), y.max())


/kaggle/input/asvpoof-2019-dataset/LA/LA/ASVspoof2019_LA_train/flac/LA_T_9552332.flac
Length: 64000
Min/Max: -1.0 0.9621887


In [ ]:
import librosa
import numpy as np

N_MELS = 80
N_MFCC = 13

def extract_features(y):

    # ---- Log-Mel Spectrogram ----
    mel = librosa.feature.melspectrogram(
        y=y,
        sr=TARGET_SR,
        n_fft=1024,
        hop_length=256,
        n_mels=N_MELS,
        power=2.0
    )

    logmel = librosa.power_to_db(mel)

    # ---- MFCC from Log-Mel ----
    mfcc = librosa.feature.mfcc(
        S=logmel,
        n_mfcc=N_MFCC
    )

    delta  = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)

    mfcc_all = np.vstack([mfcc, delta, delta2])

    return mfcc_all, logmel


In [ ]:
mfcc, logmel = extract_features(y)

print("MFCC shape:", mfcc.shape)
print("Log-Mel shape:", logmel.shape)


MFCC shape: (39, 251)
Log-Mel shape: (80, 251)
